In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

# Read the Excel file
df = pd.read_excel("staff.xlsx")

# Read the project types
df_types = pd.read_excel("types.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Create dropdown menu options for project types
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    dbc.Button("New Project", id="open", color="primary", className="mr-1"),
    dbc.Modal(
        [
            dbc.ModalHeader("New Project Details"),
            dbc.ModalBody(
                [
                    dcc.Input(id='type-input', type='text', placeholder='Type of Project'),
                    dcc.Input(id='duration-input', type='text', placeholder='Duration of Project')
                ]
            ),
            dbc.ModalFooter(
                [
                    dbc.Button("Close", id="close", className="ml-auto"),
                    dbc.Button("Save Changes", id="save-changes", className="ml-1")
                ]
            ),
        ],
        id="modal",
        size='lg',
        centered=True,
        backdrop='static'
    ),
    html.Div([
        html.Label('Type of Project:'),
        dcc.Dropdown(
            id='type-dropdown',
            options=type_options,
            value=type_options[0]['value'] if type_options else None  # Default value
        )
    ]),
    dcc.Dropdown(
        id='project-dropdown',
        options=project_options,
        value=project_options[0]['value']  # Default value
    ),
    dcc.Dropdown(
        id='job-dropdown',
        multi=True,
    ),
    dcc.Graph(id='deployment-plan'),
    html.Div(id='selected-project-type')  # Show the selected project type
])


# Define callback to update job dropdown options based on selected project
@app.callback(
    Output('job-dropdown', 'options'),
    [Input('project-dropdown', 'value')]
)
def update_job_dropdown(selected_project):
    # Filter the DataFrame based on the selected project
    filtered_data = df[df['Project'] == selected_project]

    # Get unique job titles for the selected project
    job_options = [{'label': 'Select All', 'value': 'all'}] + [{'label': job, 'value': job} for job in
                                                                filtered_data['Job'].unique()]

    return job_options


# Define callback to update the deployment plan based on selected project and job titles
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value'),
     Input('job-dropdown', 'value')]
)
def update_plan(selected_project, selected_jobs):
    # Logic to update the deployment plan based on selected project and jobs
    # ...

    # Return the updated figure
    return {'data': [], 'layout': {}}


# New callback for the modal popup
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input("save-changes", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


# Callback to show the project type for the selected project
@app.callback(
    Output('selected-project-type', 'children'),
    [Input('project-dropdown', 'value')]
)
def display_project_type(selected_project):
    # Get the type of the selected project
    project_type = df_types[df_types['Project'] == selected_project]['Type'].values[0]
    return f'Type of Project: {project_type}'


if __name__ == '__main__':
    app.run_server(debug=True)
